# Covariance matrix for _Pantheon_ +

This notebook generates the covariance matrix for pantheon plus data only (without SH0ES). Based on https://github.com/PantheonPlusSH0ES/DataRelease/blob/main/Pantheon%2B_Data/5_COSMOLOGY/cosmosis_likelihoods/Pantheon%2B_only_cosmosis_likelihood.py

In [5]:
#Import libraries
import numpy as np
import pandas as pd
import os
import git
path_git = git.Repo('.', search_parent_directories=True).working_tree_dir
os.chdir(path_git)

In [6]:
#Define the function that computes the covmat
def covmat_pantheon_plus_only(file_pantheon_plus_data, file_pantheon_plus_cov): 

    print("Loading data from {}".format(file_pantheon_plus_data))
    data = pd.read_csv(file_pantheon_plus_data,delim_whitespace=True)
    origlen = len(data)
    ww = (data['zHD']>0.01) | (np.array(data['IS_CALIBRATOR'],dtype=bool))
    zCMB = data['zHD'][ww] #use the vpec corrected redshift for zCMB 
    
    f=open(file_pantheon_plus_cov)
    line = f.readline()
    n = int(len(zCMB))
    C = np.zeros((n,n))
    ii = -1
    jj = -1
    mine = 999
    maxe = -999
    for i in range(origlen):
        jj = -1
        if ww[i]:
            ii += 1
        for j in range(origlen):
            if ww[j]:
                jj += 1
            val = float(f.readline())
            if ww[i]:
                if ww[j]:
                    C[ii,jj] = val
    f.close()
    return C

In [7]:
# Compute the Pantheon covmat
os.chdir(path_git+'/fr_mcmc/source/Pantheon_plus_shoes')
file_pantheon_plus_data = 'Pantheon+SH0ES.dat'
file_pantheon_plus_cov = 'Pantheon+SH0ES_STAT+SYS.cov'
C = covmat_pantheon_plus_only(file_pantheon_plus_data, file_pantheon_plus_cov)

Loading data from Pantheon+SH0ES.dat
1701
1701 1657


In [8]:
#Let's save the covmat
outfile = 'covmat_pantheon_plus_only.npz'
np.savez(outfile, C)